In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

### Import required transformer libraries

In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 40.3 MB/s 
     |████████████████████████████████| 596 kB 49.8 MB/s 
     |████████████████████████████████| 61 kB 503 kB/s 
     |████████████████████████████████| 895 kB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast,DistilBertForSequenceClassification
from transformers import Trainer,TrainingArguments
from transformers import DistilBertTokenizerFast, BertForMaskedLM

### Some needed liberaries

In [4]:
from sklearn.metrics import accuracy_score, f1_score
from google.colab import drive
from torch import nn
from transformers import Trainer

# from dataset import load_metric

## Read Data

In [5]:
# path = 'drive/My Drive/DataLab/sarcasm/train.En.csv'
path = 'drive/My Drive/DataLab/sarcasm/train.En.csv'
drive.mount('/content/drive')
df = pd.read_csv(path)
df = df.dropna(subset=['tweet'])

Mounted at /content/drive


## Split Test and Train

In [6]:
train, test = train_test_split(df, test_size=0.1)

In [7]:
train.head()

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
281,281,Mental shoot out ends. Silent pause. Sirens st...,1,The sirens always start just as the gunfire fi...,1.0,0.0,0.0,0.0,0.0,0.0
256,256,did you know there is a direct correlation wit...,1,Cuffing your pants too high is embarrassing an...,1.0,0.0,0.0,0.0,0.0,1.0
3126,3126,Christmas sucked. Didn't get the heelys I want...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,134,@mjborshell @garygilligan Hasn't he opened a f...,1,Javid hasn't actually opened any new hospitals...,1.0,0.0,0.0,0.0,0.0,0.0
1885,1885,omg text “pew pew” and see what happens 😮,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
test.head()

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
1735,1735,I finished my MBio in Biomedical science this ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,366,If your website still has a google plus share ...,1,You should remove your google plus share butto...,1.0,0.0,0.0,0.0,0.0,0.0
27,27,Not at all concerning that a man's just been r...,1,Very concerning that a man's just been round t...,1.0,0.0,0.0,0.0,0.0,0.0
2692,2692,Feel like pure shit just want my @depop accoun...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1253,1253,Good morning to everyone except Tristan Thomps...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract Features and Labels

In [9]:
train_tweets = train['tweet'].values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['tweet'].values.tolist()

## Split the training sample into train and validation set

In [10]:
train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels, 
                                                                    test_size=0.1,random_state=42,stratify=train_labels)

## Steps for Fine Tuning model

<ul>
<li>Prepare dataset</li>
<li>Load pretrained tokenizer,call it with dataset</li>
<li>Build Pytorch datasets with encodings</li>
<li>Load pretrained Model</li>
<li> Load Trainer and train it </li>
    Instead of Trainer we could've use native Pytorch training pipline.
</ul>

### Set Model Name

In [11]:
model_name = 'detecting-sarcasim'

## Tokenization

In [84]:
tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-cased',
                                                    num_labels=2,
                                                    loss_function_params={"weight": [0.1, 0.3]}
                                                    )

# number of labels here is 2

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
lo

<blockquote> The difference between a “fast” and a “non-fast” tokenizer is computation speed but there is no functional difference between them.
<blockqoute> FastTokenizers are implemented in Rust and are factors faster than the Python based tokenizers. Apart from that their encoding methods should behave the same. However, they are not functionally identical.</blockqoute>

In [85]:
train_encodings = tokenizer(train_tweets, truncation=True, padding=True,return_tensors = 'pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [86]:
val_encodings = tokenizer(val_tweets, truncation=True, padding=True,return_tensors = 'pt')

In [87]:
test_encodings = tokenizer(test_tweets, truncation=True, padding=True,return_tensors = 'pt')

<ul>
    <li>setting truncation = True will eliminate tokens that exceed the max_length(512) in case of BERT.</li>
    <li>setting padding =True will pad documents that have length less than max_length with empty tokens i.e. 0, ensuring that all of our sequences are padded to the same length.</li>
    <li>setting return_tensors = ‘pt’ will return the encodings as pytorch tensors.</li>
    <li>This will allow us to feed batches of sequences into the model at the same time.</li>
</ul>

## Turn labels and encodings into a Dataset object

<ul>
    <li>Wrap the tokenized data into a torch dataset.</li>
    <li>In PyTorch, this is done by subclassing a torch.utils.data.Dataset object and implementing len and getitem.</li>
<ul>   

In [88]:
class SarcasimDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
## Test Dataset
class SarcasimTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

## Genearte DataLoaders

In [89]:
train_dataset = SarcasimDataset(train_encodings, train_labels)

In [90]:
val_dataset = SarcasimDataset(val_encodings, val_labels)

In [91]:
test_dataset = SarcasimTestDataset(test_encodings)

## Define a Simple Metrics Function

In [92]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1}

In [93]:
training_args = TrainingArguments(
    output_dir='./res', evaluation_strategy="steps", num_train_epochs=5, per_device_train_batch_size=32,
    per_device_eval_batch_size=64, warmup_steps=500, weight_decay=0.01,logging_dir='./logs4',
    #logging_steps=10,
    load_best_model_at_end=True,
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


<ul>
    <li> output_dir = output directory</li>
    <li> num_train_epochs = total number of training epochs</li>
    <li> per_device_train_batch_size = batch size per device during training</li>
    <li> per_device_eval_batch_size = batch size for evaluation</li>
    <li> warmup_steps = number of warmup steps for learning rate scheduler</li>
    <li> weight_decay = strength of weight decay</li>
    <li> logging_dir = directory for storing logs</li>
<ul>

## Fine Tuning with Trainer

In [94]:
model = DistilBertForSequenceClassification.from_pretrained("bert-base-cased",num_labels=2)
# distilbert-base-uncased
# bert-base-cased

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "qa_dropou

In [95]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [96]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


<ul>
    <li> model = the instantiated hugging-face Transformers model to be trained </li>
    <li> args=training_args = training arguments, defined above</li>
    <li> train_dataset = training dataset</li>
    <li> eval_dataset = evaluation dataset</li>
<ul>

In [97]:
trainer.train()

***** Running training *****
  Num examples = 2808
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 440


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=440, training_loss=0.5536378340287642, metrics={'train_runtime': 178.0762, 'train_samples_per_second': 78.843, 'train_steps_per_second': 2.471, 'total_flos': 916304805842400.0, 'train_loss': 0.5536378340287642, 'epoch': 5.0})

In [98]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 312
  Batch size = 64


{'epoch': 5.0,
 'eval_accuracy': 0.6442307692307693,
 'eval_f1_score': 0.6620442509055567,
 'eval_loss': 0.6211642026901245,
 'eval_runtime': 1.1694,
 'eval_samples_per_second': 266.8,
 'eval_steps_per_second': 4.276}

In [99]:
# test['sarcastic'] = 0
# test_tweets = test['tweet'].values.tolist() 
# test_labels = test['sarcastic'].values.tolist() 
# test_encodings = tokenizer(test_tweets,
#                            truncation=True, 
#                            padding=True,
#                            return_tensors = 'pt').to("cuda") 
# test_dataset = SentimentDataset(test_encodings, test_labels)

## Test

In [100]:
pin_memory=False
preds = trainer.predict(test_dataset=test_dataset)

***** Running Prediction *****
  Num examples = 2
  Batch size = 64


In [101]:
probs = torch.from_numpy(preds[0]).softmax(1)

# convert tensors to numpy array
predictions = probs.numpy()

In [102]:
newdf = pd.DataFrame(predictions,columns=['Neutral_1','Positive_2'])

In [103]:
newdf.head()

,Neutral_1,Positive_2
0,0.717921,0.282079
1,0.712581,0.287419


In [104]:
def labels(x):
  if x == 0:
    return 'Negative_1'
  else:
    return 'Positive_1'

results = np.argmax(predictions,axis=1)
# test['sarcastic'] = results
test['sarcastic'] = test['sarcastic'].map(labels)
test.head()

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
1735,1735,I finished my MBio in Biomedical science this ...,Positive_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,366,If your website still has a google plus share ...,Positive_1,You should remove your google plus share butto...,1.0,0.0,0.0,0.0,0.0,0.0
27,27,Not at all concerning that a man's just been r...,Positive_1,Very concerning that a man's just been round t...,1.0,0.0,0.0,0.0,0.0,0.0
2692,2692,Feel like pure shit just want my @depop accoun...,Positive_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1253,1253,Good morning to everyone except Tristan Thomps...,Positive_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
